In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import sklearn
from skimage import io, transform
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline

# Define functions for loading/augmenting data and building/training the CNN

In [2]:
NUM_CLASSES = 6

def load_data():
    x_train = np.load('exam2_train_x.npy').astype('float32')
    y_train = np.load('exam2_train_y.npy').astype('float32')
    """
    if os.path.exists('x_train_augmented.npy') and os.path.exists('y_train_augmented.npy'):
        print "Training data has been augmented by rotating and flipping horizontally"
        x_train = np.load('x_train_augmented.npy').astype('float32')
        y_train = np.load('y_train_augmented.npy').astype('float32')
    else:
        print "Training data is not yet augmented..."
        x_train = np.load('exam2_train_x.npy').astype('float32')
        y_train = np.load('exam2_train_y.npy').astype('float32')
    """  
       
    x_test = np.load('exam2_test_x.npy').astype('float32')
    y_test = np.load('exam2_test_y.npy').astype('float32')

    return x_train, y_train, x_test, y_test

def rotate_imgs(x, y, d=20):
    """
        Makes a copy of X and Y -- image and label arrays.
        Rotates each image D degrees left and right.
        Appends these rotated images and their corresponding labels to the copied arrays.
    """
    
    new_x, new_y = np.copy(x), np.copy(y)
    for image, label in zip(x, y):
        
        image_l = transform.rotate(image, d, mode='edge', resize=False, preserve_range=True)
        image_r = transform.rotate(image, -d, mode='edge', resize=False, preserve_range=True)
        
        new_x = np.append(new_x, [image_l], axis=0)
        new_y = np.append(new_y, [label], axis=0)
        new_x = np.append(new_x, [image_r], axis=0)
        new_y = np.append(new_y, [label], axis=0)
                
    return new_x.astype('float32'), new_y

def flip_imgs(x, y):
    """
        Makes a copy of X and Y -- image and label arrays.
        Flips each image horizontally.
        Appends these flipped images and their corresponding labels to the copied arrays.
    """
    
    new_x, new_y = np.copy(x), np.copy(y)
    for image, label in zip(x, y):

        new_x = np.append(new_x, [np.fliplr(image)], axis=0)
        new_y = np.append(new_y, [label], axis=0)
    
    return new_x.astype('float32'), new_y

def conv_model(features, labels, mode): 
    input_layer = tf.reshape(features["x"], [-1, 64, 64, 3])

    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[4, 4],
        padding="same",
        activation=tf.nn.relu)
    
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    conv3 = tf.layers.conv2d(
        inputs=pool2,
        filters=128,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu)
    
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    
    pool3_flat = tf.reshape(pool3, [-1, 8 * 8 * 128])
    
    dense1 = tf.layers.dense(inputs=pool3_flat, units=1024, activation=tf.nn.relu)
    dropout1 = tf.layers.dropout(inputs=dense1, rate=0.5, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    dense2 = tf.layers.dense(inputs=dropout1, units=512, activation=tf.nn.relu)
    dropout2 = tf.layers.dropout(inputs=dense2, rate=0.5, training=mode == tf.estimator.ModeKeys.TRAIN)

    logits = tf.layers.dense(inputs=dropout2, units=NUM_CLASSES, activation=tf.nn.sigmoid)
        
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=NUM_CLASSES)
    
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
  
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

# Load in data and, if necessary, augment it by rotating and flipping images

In [3]:
x_train, y_train, x_test, y_test = load_data()

"""
if x_train.shape[0] == 1020 or y_train.shape[0] == 1020:
    print "\tLet's rotate and flip the training data images to produce an augmented training set."
    
    x_train, y_train = rotate_imgs(x_train, y_train)
    x_train, y_train = flip_imgs(x_train, y_train)
    
    print "\tAfter rotating and flipping the images of the training set, we have {} images and {} labels.".format(
        x_train.shape[0], y_train.shape[0])
    
    np.save('x_train_augmented.npy', x_train)
    np.save('y_train_augmented.npy', y_train)
"""

'\nif x_train.shape[0] == 1020 or y_train.shape[0] == 1020:\n    print "\tLet\'s rotate and flip the training data images to produce an augmented training set."\n    \n    x_train, y_train = rotate_imgs(x_train, y_train)\n    x_train, y_train = flip_imgs(x_train, y_train)\n    \n    print "\tAfter rotating and flipping the images of the training set, we have {} images and {} labels.".format(\n        x_train.shape[0], y_train.shape[0])\n    \n    np.save(\'x_train_augmented.npy\', x_train)\n    np.save(\'y_train_augmented.npy\', y_train)\n'

# Train classifier

In [4]:
clf = tf.estimator.Estimator(model_fn=conv_model, model_dir="convnet_model")

tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
clf.train(
    input_fn=train_input_fn,
    steps=1,
    hooks=[logging_hook])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_model_dir': 'convnet_model', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.24384595 0.08970606 0.08971474 0.24384595 0.24318142 0.08970591]
 [0.11395329 0.23719078 0.23714922 0.0872577  0.087258   0.237191  ]
 [0.07843921 0.20878366 0.07785791 0.21163975 0.21163975 0.21163975]
 [0.1862933  0.1862933  0.1862933  0.1862933  0.06853348 0.1862933 ]
 [0.21115923 0.07768122 0.21115945 0.21115945 0.07768124 0.21115945]
 [0.06853348 0.1862933  0.1862933  0.1862933  0.1862933  0.1862933 ]
 [0.18672445 0.06869209 0.18672445 0.18672445 0.18672445 0.18441014]
 [0.20764174 0.09304559 0.207641

# Evaluate model

In [5]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_results = clf.evaluate(input_fn=eval_input_fn)

print eval_results

INFO:tensorflow:Starting evaluation at 2018-04-14-16:10:19
INFO:tensorflow:Restoring parameters from convnet_model/model.ckpt-1
INFO:tensorflow:Finished evaluation at 2018-04-14-16:10:20
INFO:tensorflow:Saving dict for global step 1: accuracy = 0.17222223, global_step = 1, loss = 1.8654065
{'loss': 1.8654065, 'global_step': 1, 'accuracy': 0.17222223}
